In [2]:
import pandas as pd
import numpy as np
import random
import string
import matplotlib.pyplot as plt

In [3]:
keywords = pd.DataFrame(np.random.randint(low=0, high=10, size=(100, 2)),
...                    columns=['Keywords', "Timestamp"])

In [4]:
keywords = keywords.sort_values(['Timestamp'], ascending=[1])
keywords.head()

,Keywords,Timestamp
49,3,0
87,3,0
76,0,0
39,0,0
22,3,0


In [5]:
df = pd.DataFrame(np.random.randint(low=0, high=10, size=(100, 10)), columns=['time.start','time.end', 'frame.start', 'frame.end', 'transcript','frames.before','frames.after','angry.change','surprise.change','happy.change'])

#If you want random keywords
#df["keywords"] = [''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10)) for x in range(100)]

#Randomly sample from finite list of keywords
possible_keywords = ['a', 'b', 'c', 'd', 'e']
df["keywords"] = [random.choice(possible_keywords) for x in range(100)]

In [6]:
df = df.sort_values(['time.start'], ascending=[1])
df.head()

,time.start,time.end,frame.start,frame.end,transcript,frames.before,frames.after,angry.change,surprise.change,happy.change,keywords
99,0,8,2,7,8,6,6,1,5,7,b
37,0,2,3,7,3,4,0,0,3,2,a
55,0,2,9,2,1,7,8,3,3,3,d
60,0,9,6,5,8,1,4,0,6,3,c
32,0,6,7,3,0,9,6,4,8,1,b


In [32]:
df_keywords_dummies = pd.get_dummies(df)
corr_matrix = df_keywords_dummies.iloc[:,7:].corr()
corr_matrix
for x in range(0,3):
    print(np.max(corr_matrix.iloc[x,3:],axis=0))

0.0638530702555
0.105586064131
0.100110656224


,angry.change,surprise.change,happy.change,keywords_a,keywords_b,keywords_c,keywords_d,keywords_e
angry.change,1.000000,0.160299,-0.109773,0.001390,-0.084593,-0.022670,0.063853,0.033715
surprise.change,0.160299,1.000000,0.007713,-0.066842,0.068204,-0.062089,0.105586,-0.053448
happy.change,-0.109773,0.007713,1.000000,-0.034268,0.001256,0.100111,-0.084580,0.017141
keywords_a,0.001390,-0.066842,-0.034268,1.000000,-0.196818,-0.216587,-0.236067,-0.223100
keywords_b,-0.084593,0.068204,0.001256,-0.196818,1.000000,-0.241560,-0.263286,-0.248825
keywords_c,-0.022670,-0.062089,0.100111,-0.216587,-0.241560,1.000000,-0.289731,-0.273817
keywords_d,0.063853,0.105586,-0.084580,-0.236067,-0.263286,-0.289731,1.000000,-0.298444
keywords_e,0.033715,-0.053448,0.017141,-0.223100,-0.248825,-0.273817,-0.298444,1.000000


In [8]:
plt.scatter(df['Timestamp'],emotions["Happy"])

plt.ylabel('Happiness')
plt.xlabel('Timestamp')
plt.show()

KeyError: 'Timestamp'

In [98]:
for index, row in keywords.iterrows():
    

Keywords     U441A9JLZG
Timestamp             0
Name: 0, dtype: object
Keywords     PROB3BG08S
Timestamp             3
Name: 1, dtype: object
Keywords     0JI0VIK2ZF
Timestamp             2
Name: 2, dtype: object
Keywords     LHVSBRDSA6
Timestamp             5
Name: 3, dtype: object
Keywords     VDS7TF1L53
Timestamp             4
Name: 4, dtype: object
Keywords     9V0DGKERZO
Timestamp             1
Name: 5, dtype: object
Keywords     C6EOOYNR6D
Timestamp             4
Name: 6, dtype: object
Keywords     07BL64PU2K
Timestamp             4
Name: 7, dtype: object
Keywords     ABO8N8SH58
Timestamp             6
Name: 8, dtype: object
Keywords     DN9EANZUOV
Timestamp             8
Name: 9, dtype: object
Keywords     WS608W097B
Timestamp             9
Name: 10, dtype: object
Keywords     7E4WY6QTHD
Timestamp             2
Name: 11, dtype: object
Keywords     19GNYSLCZK
Timestamp             0
Name: 12, dtype: object
Keywords     TGJYQXVPY8
Timestamp             6
Name: 13, dtype: object
Ke